## SecML
For some of our experiments, we used the SecML library. SecML is an open-source Python library for the generation of adversarial examples and the evaluation of machine learning models under attack. It is developed by the NECSTLab, a research group at the Department of Electronics, Information, and Bioengineering of Politecnico di Milano. The library and documentation is available at https://secml.gitlab.io/.

The secml library has already been installed in the environment.

## The Model

In this lab, we will download the MNIST dataset and train a simple model to predict handwritten digits.

You can learn more about the MNIST dataset here: https://en.wikipedia.org/wiki/MNIST_database

In [1]:
import secml

from secml.data.loader import CDataLoaderMNIST

# MNIST dataset will be downloaded and cached if needed
loader = CDataLoaderMNIST()

---

Now, let's train the model only for digits 1, 4, 5 and 9.

In subsequent labs, we will use this model (but with just 5 & 9) to demonstrate attacks.

In [2]:
random_state = 999

n_tr = 100  # Number of training set samples
n_val = 500  # Number of validation set samples
n_ts = 500  # Number of test set samples

digits = (1, 4, 5, 9)

In [ ]:

tr_val = loader.load('training', digits=digits, num_samples=n_tr + n_val)
ts = loader.load('testing', digits=digits, num_samples=n_ts)

# Split in training and validation set
tr = tr_val[:n_tr, :]
val = tr_val[n_tr:, :]

# Normalize the features in `[0, 1]`
tr.X /= 255
val.X /= 255
ts.X /= 255

from secml.ml.classifiers import CClassifierSVM
# train SVM in the dual space, on a linear kernel, as needed for poisoning
clf = CClassifierSVM(C=10, kernel='linear')

print("Training of classifier...")
clf.fit(tr.X, tr.Y)

# Compute predictions on a test set
y_pred = clf.predict(ts.X)

# Metric to use for performance evaluation
from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

# Evaluate the accuracy of the classifier
acc = metric.performance_score(y_true=ts.Y, y_pred=y_pred)

print("Accuracy on test set: {:.2%}".format(acc))

### Visualizing the dataset
Now let's show some of the images along with their predicted labels.

We define a function `show_digits` to display the images.  We will use this again in future labs.

In [ ]:
from secml.figure import CFigure
# Only required for visualization in notebooks
%matplotlib inline

# Let's define a convenience function to easily plot the MNIST dataset
def show_digits(samples, preds, labels, digs, n_display=8):
    samples = samples.atleast_2d()
    n_display = min(n_display, samples.shape[0])
    fig = CFigure(width=n_display*2, height=3)
    for idx in range(n_display):
        fig.subplot(2, n_display, idx+1)
        fig.sp.xticks([])
        fig.sp.yticks([])
        fig.sp.imshow(samples[idx, :].reshape((28, 28)), cmap='gray')
        fig.sp.title("{} ({})".format(digits[labels[idx].item()], digs[preds[idx].item()]),
                     color=("green" if labels[idx].item()==preds[idx].item() else "red"))
    fig.show()

show_digits(ts.X, clf.predict(ts.X), ts.Y, digits)


The results of this lab demonstrate that the model is able to predict the digits with high accuracy as shown above.

We will use this model in subsequent labs to demonstrate adversarial attacks.